In [1]:
import numpy as np
import math  
from sklearn.cluster import KMeans

In [2]:
from preprocessing import getData

In [3]:
def kernelFunction(XminusMu, sigmaK, name):
    phi = 0.0
    if sigmaK == 0: sigmaK = 1e-10 #This step is because when one of the cluster has only value that means its sigma=0 
    if name == "Gaussian":
        phi = np.exp((-0.5 * (XminusMu**2))/(sigmaK**2))
    elif name == "Multiquadratic":
        phi = (XminusMu**2 + sigmaK**2)**0.5 
    elif name == "Linear":
        phi = XminusMu
    return phi

In [4]:
def train(train_X, train_Y, K, name):
    m = train_X.shape[0]
    classes = train_Y.shape[1]
    # K Means on train_X
    # Number of hidden neurons = Number of clusters K
    # mu(k) = centroid of kth cluster (k = 1 to K)
    kmeans = KMeans(n_clusters = K, max_iter=1000, random_state = 0).fit(train_X) 
    mu = kmeans.cluster_centers_
    labels = kmeans.predict(train_X)
    # sigma(k) = 1/m(k) * sum over i=1 to m(k) [||X(i) - Mu(k)||]
    sigma = np.zeros(K)
    for k in range(K):
        clusterK = train_X[(labels == k)]
        mK = len(clusterK)
        summation = 0.0
        for i in range(mK):
            summation = summation + np.linalg.norm(clusterK[i]-mu[k],1)
        sigma[k] = (1.0/mK) * summation
    # Evaluate hidden layer matrix H m*K
    H = np.ndarray((m, K))
    # H[i][k] = phi(||X(i) - mu(k)||)
    for i in range(m):
        for k in range(K):
            XminusMu = np.linalg.norm(train_X[i] - mu[k], 1)
            H[i][k] = kernelFunction(XminusMu, sigma[k], name)
    # Weight matrix W = H^-1 Y () m*K.K*classes = m*classes
    W = np.dot(np.linalg.pinv(H),train_Y)
    #Return Values
    return [W, mu, sigma, name]

In [5]:
def test(test_X, test_Y, W, K, mu, sigma, name):
    # Evaluate hidden layer matrix H m*K
    m = test_X.shape[0]
    H = np.zeros((m, K))
    # H[i][k] = phi(||X(i) - mu(k)||)
    for i in range(m):
        for k in range(K):
            XminusMu = np.linalg.norm(test_X[i] - mu[k], 1)
            H[i][k] = kernelFunction(XminusMu, sigma[k], name)
    # Y_predicted = H.dot(W)
    Y_predicted = H.dot(W)
    # MaxIndex of Y_predicted[i] is prediction for test_X[i] and compare with test_Y[i]
    count = 0
    for i in range(m):
        actualClass = np.argmax(test_Y[i])
        predictedClass = np.argmax(Y_predicted[i])
        if actualClass == predictedClass:
            count += 1
    print(count/m*100)

In [6]:
X, Y = getData('data.mat')
#Holdout method -> 
def holdout(X, Y, train_percent, K, name):
    train_size = int(train_percent*X.shape[0])
    train_X = X[:train_size,:]
    test_X = X[train_size:,:]
    train_Y = Y[:train_size,:]
    test_Y = Y[train_size:,:]
    [W, mu, sigma, name] = train(train_X, train_Y, K, name)
    return test(test_X, test_Y, W, K, mu, sigma, name)

In [7]:
holdout(X, Y, 0.7, 350, "Gaussian")
holdout(X, Y, 0.7, 350, "Multiquadratic")
holdout(X, Y, 0.7, 350, "Linear")

93.02325581395348
89.30232558139535
90.23255813953487


In [ ]:
gAcc = mAcc = lAcc = 0.0
gK = mK = lK = 0
for i in range(150, 600, 10):
    a1 = holdout(X, Y, 0.7, i, "Gaussian")
    if gAcc < a1:
        gAcc = a1
        gK = i
    a2 = holdout(X, Y, 0.7, i, "Multiquadratic")
    if mAcc < a2:
        mAcc = a2
        mK = i
    a3 = holdout(X, Y, 0.7, i, "Linear")
    if lAcc < a3:
        lAcc = a3
        lK = i

In [13]:
gAcc

95.50387596899225

In [14]:
mAcc

93.48837209302326

In [15]:
lAcc

93.33333333333333

In [17]:
gK, mK, lK

(370, 460, 370)

In [26]:
holdout(X, Y, 0.7, 350, "Gaussian")
holdout(X, Y, 0.7, 460, "Multiquadratic")
holdout(X, Y, 0.7, 370, "Linear")

94.72868217054263
92.55813953488372
93.1782945736434


In [27]:
holdout(X, Y, 0.7, 350, "Gaussian")
holdout(X, Y, 0.7, 350, "Multiquadratic")
holdout(X, Y, 0.7, 350, "Linear")

94.72868217054263
92.4031007751938
91.93798449612403
